In [1]:
import sys
sys.path.append("..")
import torch
from torch import nn
from torchvision import transforms
from dataloaders.FashionMnist import load_FashionMnist
from tools.model_trainer import train_model
from tools.model_tester import test_model
from torchsummary import summary
from easydict import EasyDict as edict

# 1 加载数据集

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = edict({
    'device':device,
    'batch_size': 64,
    'num_classes': 10,  # 分类类别
    'lr': 0.01,  # 学习率
    'epoch_size': 20  # 训练次数
})

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])
train_iter, _, test_iter = load_FashionMnist(batch_size=cfg.batch_size, transform=transform)
print(f"train size: {len(train_iter.dataset)}")
# print(f"valid size: {len(valid_iter.dataset)}")
print(f"test size: {len(test_iter.dataset)}")
for X, y in test_iter:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

train size: 60000
test size: 10000
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


d:\Anaconda3\envs\lxh_torch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 2 不使用正则化的卷积神经网络

In [6]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr)
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_without_normalization') 

training on cuda
Epoch 1/20
937/937 [========================================] Train Loss: 0.4992, Train Accuracy: 0.8114, Valid Loss: 0.4258, Valid Accuracy: 0.8356, Cost Time 13.3510 sec
Epoch 2/20
937/937 [========================================] Train Loss: 0.3590, Train Accuracy: 0.8688, Valid Loss: 0.3562, Valid Accuracy: 0.8704, Cost Time 11.1327 sec
Epoch 3/20
937/937 [========================================] Train Loss: 0.3381, Train Accuracy: 0.8759, Valid Loss: 0.3667, Valid Accuracy: 0.8661, Cost Time 11.7559 sec
Epoch 4/20
937/937 [========================================] Train Loss: 0.3241, Train Accuracy: 0.8812, Valid Loss: 0.3733, Valid Accuracy: 0.8623, Cost Time 15.1042 sec
Epoch 5/20
937/937 [========================================] Train Loss: 0.3175, Train Accuracy: 0.8854, Valid Loss: 0.3766, Valid Accuracy: 0.8664, Cost Time 16.9977 sec
Epoch 6/20
937/937 [========================================] Train Loss: 0.3073, Train Accuracy: 0.8880, Valid Loss: 0.364

# 3 对比正则化技术

## 3.1 使用L1正则化

In [3]:
from tools.model_trainer import train_batch_L1

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=0.001) # L2正则化
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, train_func=train_batch_L1, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_L1_normalization') 

training on cuda
Epoch 1/20
0/937 [               ]

d:\Anaconda3\envs\lxh_torch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


937/937 [===============] Train Loss: 1.0921, Train Accuracy: 0.7128, Valid Loss: 0.5580, Valid Accuracy: 0.7894, Cost Time 25.2613 sec
Epoch 2/20
937/937 [===============] Train Loss: 0.8279, Train Accuracy: 0.8124, Valid Loss: 0.5397, Valid Accuracy: 0.8116, Cost Time 22.2559 sec
Epoch 3/20
937/937 [===============] Train Loss: 0.7806, Train Accuracy: 0.8218, Valid Loss: 0.4863, Valid Accuracy: 0.8201, Cost Time 19.1095 sec
Epoch 4/20
937/937 [===============] Train Loss: 0.7598, Train Accuracy: 0.8274, Valid Loss: 0.4796, Valid Accuracy: 0.8223, Cost Time 15.6250 sec
Epoch 5/20
937/937 [===============] Train Loss: 0.7571, Train Accuracy: 0.8307, Valid Loss: 0.5247, Valid Accuracy: 0.8115, Cost Time 16.5656 sec
Epoch 6/20
937/937 [===============] Train Loss: 0.7352, Train Accuracy: 0.8352, Valid Loss: 0.4941, Valid Accuracy: 0.8250, Cost Time 19.0371 sec
Epoch 7/20
937/937 [===============] Train Loss: 0.7413, Train Accuracy: 0.8317, Valid Loss: 0.4820, Valid Accuracy: 0.8240, Cost

## 3.2 使用L2正则化

In [4]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=0.001) # L2正则化
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_L2_normalization') 

training on cuda
Epoch 1/20
937/937 [========================================] Train Loss: 0.6601, Train Accuracy: 0.7496, Valid Loss: 0.5170, Valid Accuracy: 0.8025, Cost Time 13.3259 sec
Epoch 2/20
937/937 [========================================] Train Loss: 0.5167, Train Accuracy: 0.8127, Valid Loss: 0.4991, Valid Accuracy: 0.8115, Cost Time 11.6148 sec
Epoch 3/20
937/937 [========================================] Train Loss: 0.4957, Train Accuracy: 0.8214, Valid Loss: 0.4950, Valid Accuracy: 0.8226, Cost Time 11.2439 sec
Epoch 4/20
937/937 [========================================] Train Loss: 0.4919, Train Accuracy: 0.8228, Valid Loss: 0.5294, Valid Accuracy: 0.8089, Cost Time 11.1479 sec
Epoch 5/20
937/937 [========================================] Train Loss: 0.4827, Train Accuracy: 0.8257, Valid Loss: 0.4941, Valid Accuracy: 0.8198, Cost Time 11.9004 sec
Epoch 6/20
937/937 [========================================] Train Loss: 0.4789, Train Accuracy: 0.8266, Valid Loss: 0.504

## 3.3 使用Dropout正则化

In [5]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_Dropout') 

training on cuda
Epoch 1/20
937/937 [========================================] Train Loss: 0.7863, Train Accuracy: 0.7073, Valid Loss: 0.5043, Valid Accuracy: 0.7973, Cost Time 14.1354 sec
Epoch 2/20
937/937 [========================================] Train Loss: 0.6276, Train Accuracy: 0.7825, Valid Loss: 0.4826, Valid Accuracy: 0.8348, Cost Time 12.5853 sec
Epoch 3/20
937/937 [========================================] Train Loss: 0.6002, Train Accuracy: 0.7972, Valid Loss: 0.5037, Valid Accuracy: 0.8261, Cost Time 12.1580 sec
Epoch 4/20
937/937 [========================================] Train Loss: 0.5893, Train Accuracy: 0.8003, Valid Loss: 0.4675, Valid Accuracy: 0.8439, Cost Time 11.0579 sec
Epoch 5/20
937/937 [========================================] Train Loss: 0.5976, Train Accuracy: 0.8011, Valid Loss: 0.4450, Valid Accuracy: 0.8396, Cost Time 10.3042 sec
Epoch 6/20
937/937 [========================================] Train Loss: 0.5817, Train Accuracy: 0.8059, Valid Loss: 0.499

## 3.4 使用BN正则化

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.BatchNorm2d(6), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.BatchNorm2d(6), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_BN') 

## 3.5 使用Layer Normalization正则化

In [7]:
summary(net, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         MaxPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
              ReLU-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
             ReLU-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

In [8]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.LayerNorm((6, 28, 28)), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.LayerNorm((16, 10, 10)), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_LN') 

training on cuda
Epoch 1/20
937/937 [===============] Train Loss: 1.0180, Train Accuracy: 0.5984, Valid Loss: 0.5627, Valid Accuracy: 0.8046, Cost Time 15.6610 sec
Epoch 2/20
937/937 [===============] Train Loss: 0.7480, Train Accuracy: 0.7194, Valid Loss: 0.5063, Valid Accuracy: 0.7988, Cost Time 13.5620 sec
Epoch 3/20
937/937 [===============] Train Loss: 0.6704, Train Accuracy: 0.7571, Valid Loss: 0.4864, Valid Accuracy: 0.8183, Cost Time 17.8880 sec
Epoch 4/20
937/937 [===============] Train Loss: 0.6263, Train Accuracy: 0.7769, Valid Loss: 0.5092, Valid Accuracy: 0.7829, Cost Time 15.9799 sec
Epoch 5/20
937/937 [===============] Train Loss: 0.6016, Train Accuracy: 0.7861, Valid Loss: 0.4777, Valid Accuracy: 0.8392, Cost Time 19.4339 sec
Epoch 6/20
937/937 [===============] Train Loss: 0.5832, Train Accuracy: 0.7958, Valid Loss: 0.4512, Valid Accuracy: 0.8370, Cost Time 20.6814 sec
Epoch 7/20
937/937 [===============] Train Loss: 0.5712, Train Accuracy: 0.8004, Valid Loss: 0.4592, 

## 3.6 SELU

In [9]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.SELU(),
    nn.Linear(120, 84), nn.SELU(),
    nn.Linear(84, cfg.num_classes)
)

# 默认是Kaiming初始化

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr)
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_SELU') 

training on cuda
Epoch 1/20
107/937 [=              ]

KeyboardInterrupt: 